In [60]:
import twitter

In [61]:
# initialize api instance
twitter_api = twitter.Api(consumer_key='8sKFgj4r3aCfmvHLNvKTLuS2A',
                        consumer_secret='qqeo83EDXzX3YqMc8wC5HzeSxDn7nJoOJpP11ExIa8Ediyj3RC',
                        access_token_key='4276054402-ghsIYrOMEVHuApV5XHtR4qEQidNGIa50aQAhvLp',
                        access_token_secret='waTDjk0TkM66Nq32rGpd4v2zdJRf6TaMAHfNpaPxBsqUl')

In [62]:
# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Wed Nov 25 16:03:12 +0000 2015", "default_profile": true, "description": "LinkedIn - https://t.co/EQ2RuxvRM0\nGraduate Student at Texas A&M University. Mays Business School. Management Information Systems", "favourites_count": 187, "followers_count": 13, "friends_count": 53, "id": 4276054402, "id_str": "4276054402", "name": "Rohit", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/4276054402/1448472147", "profile_image_url": "http://pbs.twimg.com/profile_images/669548138349395968/ZgpBe8kD_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/669548138349395968/ZgpBe8kD_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", 

In [63]:
# For downloading tweets on a keyword (limit 180 every 15 mins)

def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [64]:
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:USA
Fetched 100 tweets for the term USA
[{'text': 'My decision to appoint @usairforce General Charles Brown as the USA’s first-ever African American military service… https://t.co/XuyZT29O55', 'label': None}, {'text': 'We don’t have a policing problem in the USA, we have a parenting problem.', 'label': None}, {'text': 'The United States stands with the United Kingdom against the Chinese Communist Party’s coercive bullying tactics. W… https://t.co/KxPtWbiaPm', 'label': None}, {'text': 'RT @icretinoreal: O mundo ta violento assim pq\nninguém usa mais a pulseirinha do reggae https://t.co/9oRJd4vyfh', 'label': None}]


In [ ]:
def buidTrainingSet(corpusFile, tweetDataFile):
    import csv
    import time
    
    corpus = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            
    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"].encode('utf8') , tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [ ]:
corpusFile = "corpus.csv"
tweetDataFile = "tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile)

In [ ]:
# Reading Training Dataset from File

trainingData = []
import csv  
count = 0

with open('tweetDataFile.csv','r') as csvfile:
    lineReader = csv.reader(csvfile, delimiter=',')
    for row in lineReader:
        if row != []:
            trainingData.append({"tweet_id":row[0], "text":row[1], "label":row[2] ,"topic":row[3]})     

## Pre-processing Tweets in The Data Sets

In [72]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

[nltk_data] Downloading package punkt to C:\Users\Rohit
[nltk_data]     Jadhav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Rohit
[nltk_data]     Jadhav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [133]:
class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames 
        #tweet = re.sub('''[,b']''' ,'', tweet) # remove b' and b" ***** Try to remove this noise*******
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello) ***Wrong****
        return [word for word in tweet if word not in self._stopwords]

In [137]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

## Naive Bayes Classifier

### Building the vocabulary

### Matching tweets against our vocabulary